# SBIRT GRPA Coding

Goverment Perfromance and Results Act (GPRA) coding for intake data of recovery specialist for the Institure for Prevention and Recovery (IFPR) at RWJ Baranabas

In [1]:
#importing pandas
import pandas as pd
import numpy as np
from datetime import datetime as dt
import string as str

In [2]:
# importing data and parsing datetime columns
df = pd.read_csv(r"\\cdchomedir01\home1\tyogarro\GPRA\SBIRT_GPRA_Coding_File.csv", parse_dates = ['ADMIT_DT','FORM_DTTM',
                                                                                                'FORM_UPDATE_DTTM','DOB'])


#subsetting to include Peer Recovery Program episodes
df = df[df['EPISODE'] =='IFPR - SBIRT'].sort_values(by = ['FORM_DTTM','MRN']) # <- sorting df

In [3]:
#Renaming column to create ClientID
df.rename(columns = {'CSN':'ClientID'}, inplace = True)

In [4]:
# function to return all values greater than 30 as 30 to be utilized for days columns
def thirty(x):
    if x > 30:
        return 30 #-- returns value 30 if number greater than 30
    else:
        return x #-- will return same value if number not greater than 30

In [5]:
#Creating function for interview type
def interview_type_code(row):
    if row['INTERVIEW_TYPE'] == 'Intake':
        return 1
    elif row['INTERVIEW_TYPE'] == '6-month follow-up':
        return 2
    elif row['INTERVIEW_TYPE'] == '3-month follow-up':
        return 4
    elif row['INTERVIEW_TYPE'] == 'Discharge':
        return 5
    
#Creating new column via applying interview_type_code function to values in interview type column
df['InterviewType'] = df.apply(interview_type_code, axis = 1)

In [6]:
df['InterviewType'].value_counts()

1.0    7
5.0    5
2.0    1
Name: InterviewType, dtype: int64

## Creating GPRA 6

In [7]:
# Creating function to create column FLWPClientContacted
def FLWP_ClientContacted (row):
    if row['CLIENT_ABLE_TO_BE_CONTACTED_FOR_FOLLOW_UP'] == 'Yes':
        return 1
    elif row['CLIENT_ABLE_TO_BE_CONTACTED_FOR_FOLLOW_UP'] == 'No':
        return 0
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 2:
        return FLWP_ClientContacted (row)
    else:
        pass
    
    
#Creating column FLWPClientContacted via applying the function FLWP_ClientContacted (x)
df['FLWPClientContacted'] = df.apply(interview_type, axis = 1)

In [8]:
#Creating function to create column ReceivingServices
def Receiving_Services (row):
    if row['CLIENT_STILL_RECEIVING_PROGRAM_SERVICES'] == 'Yes':
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 2:
        return Receiving_Services (row)
    else:
        pass
    
#Creating column ReceivingServices via applying the function Receiving_Services (x)
df['ReceivingServices'] = df.apply(interview_type, axis = 1)

In [9]:
#Creating dictionary to rename columns
dictionary10 = {'FOLLOW_UP_STATUS_OF_CLIENT':'FLWPStatus','OTHER_FOLLOW_UP_STATUS':'FLWPStatusSpec',
                'CLIENT_DISCHARGE_DATE':'DischargeDate','CLIENT_DISCHARGE_STATUS':'DischargeStatusCompl',
                'REASON_FOR_TERMINATION':'DischargeStatusTermReason',
                'OTHER_REASON_FOR_TERMINATION':'OtherDischargeStatTermRsnSpec',
                'HIV_TEST_ORDERED_BY_PROGRAM':'jHIVTest','CLIENT_REFERRERD_FOR_HIV_TEST':'jHIVTestResult'}

#Renaming columns
df.rename(columns = dictionary10,inplace = True)

#Creating variables to replace null values
null_col10 = ['jHIVTest','jHIVTestResult']

#Replacing null values with -9
df[null_col10] = df[null_col10].fillna(-9)

# Data Cleaning
df['FLWPStatus'] = df['FLWPStatus'].str[:2]
# df['FLWPStatusSpec'] = df['FLWPStatusSpec'].str.title()
df['DischargeStatusCompl'] = df['DischargeStatusCompl'].str[1:2]
# df['DischargeStatusTermReason'] = df['DischargeStatusTermReason'].str[1]
# df['OtherDischargeStatTermRsnSpec'] = df['OtherDischargeStatTermRsnSpec'].str.title()
df['jHIVTest'] = df['jHIVTest'].replace('No',0)
df['jHIVTestResult'] = df['jHIVTestResult'].replace('Yes',1)
df['jHIVTestResult'] = df['jHIVTestResult'].replace('No',0)

#Converting df['DischargeDate'] to datetime data type
df['DischargeDate'] = pd.to_datetime(df['DischargeDate'])

#C,overting to the mm/dd/yyyy format
df['DischargeDate'] = df['DischargeDate'].dt.strftime('%m/%d/%Y')

In [10]:
def hiv_param(row):
    if row['jHIVTest'] == 0:
        return row['jHIVTestResult']
    elif row['jHIVTest'] == 1:
        return -1
    elif row['jHIVTest'] == -9:
        return -1
    else:
        return -9
    
df['jHIVTestResult'] = df.apply(hiv_param, axis =1 )

In [11]:
def dischargeStatusTermReason(row):
    if row['DischargeStatusCompl'] == '1':
        return -1
    elif row['DischargeStatusCompl'] == '2':
        return row['DischargeStatusTermReason']

df['DischargeStatusTermReason'] = df.apply(dischargeStatusTermReason, axis = 1)

In [12]:
# filtering FLWPStatus column based on interview type
def interview_type(row):
    if row['InterviewType'] == 2:
        return row['FLWPStatus']
    else:
        pass
    
df['FLWPStatus'] = df.apply(interview_type, axis = 1)

In [13]:
def FLWPStatus (row):
    if row['FLWPClientContacted'] == 1:
        return row['FLWPStatus']
    elif row['FLWPClientContacted'] == 0:
        return 32
    
def interview_type(row):
    if row['InterviewType'] == 2:
        return FLWPStatus (row)
    else:
        pass
    
df['FLWPStatus'] = df.apply(interview_type, axis = 1)

In [14]:
def FLWPStatusSpec (row):
    if row['FLWPStatus'] == 32:
        return 'None'
    else:
        pass
    
def interview_type(row):
    if row['InterviewType'] == 2:
        return  FLWPStatusSpec (row)
    else:
        pass
    
df['FLWPStatusSpec'] = df.apply(interview_type, axis = 1)

In [15]:
#Replacing null values with -9
df['PROGRAM_PROVIDE_NALOXONE_OR_FENTANYL_TEST_STRIPS'] = df['PROGRAM_PROVIDE_NALOXONE_OR_FENTANYL_TEST_STRIPS'].fillna(-9)

#Creating function to create column ODPreventionToolsProvided
def ODPrevention_ToolsProvided (x):
    if x == 'Neither':
        return 4
    elif x == 'Naloxone':
        return 1
    elif x == -9:
        return -9
    
#Creating column ODPreventionToolsProvided via applying the function ODPrevention_ToolsProvided (x)
df['ODPreventionToolsProvided'] = df['PROGRAM_PROVIDE_NALOXONE_OR_FENTANYL_TEST_STRIPS'].apply(lambda x: ODPrevention_ToolsProvided (x))

In [16]:
#Replacing null values with -9
df['CLIENT_FULLY_VACCINATED_AGAINST_COVID-19'] = df['CLIENT_FULLY_VACCINATED_AGAINST_COVID-19'].fillna(-9)

#Creating function to create column VaccinatedStatusCOVID19
def Vaccinated_StatusCOVID19 (x):
    if x == 'Yes':
        return 1
    elif x == 'Refused to answer':
        return -7
    elif x == 'No, client refused vaccination':
        return 4
    elif x == 'No, partially vaccinated with no plan to receive the subsequent vaccination':
        return 3
    elif x == -9:
        return -9
    
#Creating column VaccinatedStatusCOVID19 via applying the function Vaccinated_StatusCOVID19 (x)
df['VaccinatedStatusCOVID19'] = df['CLIENT_FULLY_VACCINATED_AGAINST_COVID-19'].apply(lambda x:Vaccinated_StatusCOVID19 (x))

In [17]:
#Creating column SvcInpatient
df['SvcInpatient'] = 0

#Creating column SvcOutpatient
df['SvcOutpatient'] = 0


In [18]:
#Creating columns in which no services were provided
df['SvcMethadone'] = 0
df['SvcBuprenorphine'] = 0
df['SvcNaltrexoneShortActing'] = 0
df['SvcNaltrexoneLongActing'] = 0
df['SvcDisulfiram'] = 0
df['SvcAcamprosate'] = 0
df['SvcNicotineReplacement'] = 0
df['SvcBupropion'] = 0
df['SvcVarenicline'] = 0
df['SvcResidentialRehab'] = 0
df['SvcHospitalInpatient'] = 0
df['SvcFreeStandingRes'] = 0
df['SvcAmbulatoryDetox'] = 0
df['SvcAfterCare'] = 0

In [19]:
#Creating columns with blank values
df['SvcOtherModalities'] = 0
df['SvcOtherModalitesSpec'] = " "

In [20]:
#Creating column SvcBriefTreatment
df['SvcBriefTreatment'] = 0

In [21]:
#Creating dictionary to rename columns 
dictionary8 = {'REFERRAL_TO_TREATMENT_NUMBER_OF_SESSIONS_RECEIVED':'SvcReferralTreatment', 'ASSESSMENT_NUMBER_OF_SESSIONS_RECEIVED':'SvcAssessment', 
               'TREATMENT_PLANNING_NUMBER_OF_SESSIONS_RECEIVED':'SvcTreatmentPlanning', 
               'INDIVIDUAL_COUNSELING_NUMBER_OF_SESSIONS_RECEIVED':'SvcIndividualCouns'}

#Renaming 
df.rename(columns = dictionary8,inplace = True)

#Creating variable to replace null values
col_null14 = ['SvcReferralTreatment', 'SvcTreatmentPlanning', 'SvcIndividualCouns', 'SvcAssessment']

#Replacing null values with 0
df[col_null14] = df[col_null14].fillna(0)



In [22]:
#Creating columns with missing values 
df['SvcGroupCouns'] = 0
df['SvcContingencyManagement'] = 0
df['SvcCommunityReinforcement'] = 0
df['SvcCBT'] = 0
df['SvcFamilyMarriageCouns'] = 0
df['SvcCoOccurring'] = 0
df['SvcPharmacological'] = 0
df['SvcHIVAIDSCouns'] = 0
df['SvcCulturalInterventions'] = 0
df['SvcOtherClinicalCouns'] = 0
df['SvcOtherClinicalCounsSpec'] = " "
df['SvcChildCare'] = 0

In [23]:
#Creating dictionary to rename
dictionary11 = {'CASE_MANAGEMENT_NUMBER_OF_DAYS_RECEIVED':'SvcCaseManagement', 
                'INTENSIVE_OUTPATIENT_TREATMENT_NUMBER_OF_DAYS_RECEIVED':'SvcIntensiveOutpatient', 
                'OUTREACH_NUMBER_OF_DAYS_RECEIVED':'SvcOutreach',
                'RECOVERY_SUPPORT_NUMBER_OF_DAYS_RECEIVED':'SvcRecoverySupport',
               'SCREENING_NUMBER_OF_SESSIONS_RECEIVED':'SvcScreening',
                'BRIEF_INTERVENTION_NUMBER_OF_SESSIONS_RECEIVED':'SvcBriefIntervention',
               'FAMILY_SERVICES_NUMBER_OF_SESSIONS_RECEIVED':'SvcFamilyServices', 
                'PRE_EMPLOYMENT_NUMBER_OF_SESSIONS_RECEIVED':'SvcPreEmployment'}

#Renaming columns 
df.rename(columns = dictionary11,inplace = True)

#Replacing null values with 0
col_fill0 =['SvcCaseManagement', 'SvcIntensiveOutpatient', 'SvcOutreach','SvcRecoverySupport',
            'SvcScreening','SvcBriefIntervention','SvcFamilyServices', 'SvcPreEmployment']

df[col_fill0] = df[col_fill0].fillna(0)

In [24]:
#Renaming columns 
df.rename(columns = {'EMPLOYMENT_COACHING_NUMBER_OF_SESSIONS_RECEIVED':'SvcEmploymentCoaching', 
                     'INDIVIDUAL_SERVICES_COORDINATION_NUMBER_OF_SESSIONS_RECEIVED':'SvcIndividualCoord', 
                     'TRANSPORTATION_NUMBER_OF_SESSIONS_RECEIVED': 'SvcTransportation'}, inplace = True)

#Creating variable for columns
col1 = ['SvcEmploymentCoaching','SvcIndividualCoord','SvcTransportation']

#Replacing null values
df[col1] = df[col1].fillna(0)

In [25]:
#Creating column with missing values
df['SvcHIVAIDSServices'] = 0

df.rename(columns = {'TRANSITIONAL_DRUG_FREE_HOUSING_NUMBER_OF_SESSIONS_RECEIVED':'SvcDrugFreeHousing','HOUSING_SUPPORT_NUMBER_OF_SESSIONS_RECEIVED':'SvcHousingSupport',
                     'HEALTH_INSURANCE_ENROLLMENT_NUMBER_OF_SESSIONS_RECEIVED':'SvcHealthInsuranceEnrollment',
                     'OTHER_CASE_MANAGEMENT_SERVICES_NUMBER_OF_SESSIONS_RECEIVED':'SvcOtherCaseMgmt',
                     'OTHER_CASE_MANAGEMENT_SERVICES_RECEIVED': 'SvcOtherCaseMgmtSpec'}, inplace = True)

#Creating variable 'col' to replace missing values
col2 = ['SvcDrugFreeHousing','SvcHousingSupport','SvcHealthInsuranceEnrollment','SvcOtherCaseMgmt']

#Replacing missing values with 0
df[col2] = df[col2].fillna(0)

#Editing string values
# df['SvcOtherCaseMgmtSpec'] = df['SvcOtherCaseMgmtSpec'].str.title()


In [26]:
#Renaming columns 
df.rename(columns = {'MEDICAL_CARE_NUMBER_OF_SESSIONS_RECEIVED':'SvcMedicalCare'}, inplace = True)

#Replacing missing values with 0
df['SvcMedicalCare'] = df['SvcMedicalCare'].fillna(0) 

#Creating columns with missing values 
df['SvcAlcoholDrugTesting'] = 0
df['SvcOBGYN'] = 0
df['SvcHIVAIDSMedical'] = 0
df['SvcHepatitisSupport'] = 0
df['SvcOtherSTISupport'] = 0

In [27]:
#Renaming columns
df.rename(columns = {'DENTAL_CARE_NUMBER_OF_SESSIONS_RECEIVED':'SvcDental', 
                     'OTHER_MEDICAL_SERVICES_NUMBER_OF_SESSIONS_RECEIVED':'SvcOtherMedical',
                     'OTHER_MEDICAL_SERVICES_RECEIVED':'SvcOtherMedicalSpec',
                     'CONTINUING_CARE_NUMBER_OF_SESSIONS_RECEIVED':'SvcContinuingCare',
                     'RELAPSE_PREVENTION_NUMBER_OF_SESSIONS_RECEIVED':'SvcRelapsePrevention'}, inplace = True)

#Creating variable col1 to replace missing values with 0
col3 = ['SvcDental','SvcOtherMedical','SvcContinuingCare','SvcRelapsePrevention']


#Replacing missing values with 0
df[col3] = df[col3].fillna(0) 

#Data Cleaning
# df['SvcOtherMedicalSpec'] = df['SvcOtherMedicalSpec'].str.title() 

In [28]:
#Renaming columns
df.rename(columns = {'RECOVERY_COACHING_NUMBER_OF_SESSIONS_RECEIVED':'SvcRecoveryCoaching','SELF_HELP_MUTUAL_SUPPORT_GROUP_NUMBER_OF_SESSIONS_RECEIVED':'SvcSelfHelpSupport','SPIRITUAL_SUPPORT_NUMBER_OF_SESSIONS_RECEIVED':'SvcSpiritualSupport','OTHER_AFTER_CARE_SERVICES_NUMBER_OF_SESSIONS_RECEIVED':'SvcOtherAfterCare','OTHER_AFTER_CARE_SERVICES_RECEIVED':'SvcOtherAfterCareSpec'}, inplace = True)

#Creating variable col2 to replace missing values
col4 = ['SvcRecoveryCoaching','SvcSelfHelpSupport','SvcSpiritualSupport','SvcOtherAfterCare']

#Replacing missing values with 0
df[col4] = df[col4].fillna(0)

In [29]:
#Renaming column
df.rename(columns = {'SUBSTANCE_MISUSE_EDUCATION_SERVICES_NUMBER_OF_SESSIONS_RECEIVED':'SvcSubstanceAbuseEdu'}, inplace = True )

#Replacing missing values with 0
df['SvcSubstanceAbuseEdu'] = df['SvcSubstanceAbuseEdu'].fillna(0) 

df['SvcHIVAIDSEdu'] = 0
df['SvcHepatitisEdu'] = 0
df['SvcOtherSTIEdu'] = 0
df['SvcNaloxoneTraining'] = 0
df['SvcFentanylTestStripTraining'] = 0
df['SvcOtherEdu'] = 0
df['SvcOtherEduSpec'] = " "

In [30]:
#Creating copy of RECOVERY_HOUSING_NUMBER_OF_SESSIONS_RECEIVED column
df['RECOVERY_PLANNING_NUMBER_OF_SESSIONS_RECEIVED_1'] = df['RECOVERY_PLANNING_NUMBER_OF_SESSIONS_RECEIVED']

#Renaming columns 
df.rename(columns = {'PEER_COACHING_OR_MENTORING_NUMBER_OF_SESSIONS_RECEIVED':'SvcPeerCoaching', 
                     'VOCATIONAL_SERVICES_NUMBER_OF_SESSIONS_RECEIVED':'SvcVocational',
                     'RECOVERY_HOUSING_NUMBER_OF_SESSIONS_RECEIVED':'SvcRecoveryHousing',
                     'RECOVERY_PLANNING_NUMBER_OF_SESSIONS_RECEIVED':'SvcRecoveryPlanningRSS',
                     'RECOVERY_SUPPORT_SPECIFIC_CASE_MANAGEMENT_SERVICES_NUMBER_OF_SESSIONS_RECEIVED':'SvcRecoveryCaseManagement',
                     'ALCOHOL_AND_DRUG_FREE_SOCIAL_ACTIVITIES_NUMBER_OF_SESSIONS_RECEIVED':'SvcDrugFreeSocial'}, inplace = True)

#Creating variable col3
col5 = ['SvcPeerCoaching','SvcVocational','SvcRecoveryHousing','SvcRecoveryCaseManagement','SvcDrugFreeSocial'] 

#Replacing missing values with 0
df[col5] = df[col5].fillna(0)

#inputting values in the 'SvcRecoveryPlanningRSS' column
df['SvcRecoveryPlanningRSS'] = df['SvcRecoveryPlanningRSS'].fillna(0)

In [31]:
#Renaming columns
df.rename(columns = {'INFORMATION_AND_REFERRAL_NUMBER_OF_SESSIONS_RECEIVED':'SvcInformationReferral','OTHER_RECOVERY_SUPPORT_NUMBER_OF_SESSIONS_RECEIVED':'SvcOtherRecoverySupport','OTHER_RECOVERY_SUPPORT_SERVICES_RECEIVED':'SvcOtherRecoverySupportSpec','OTHER_PEER_TO_PEER_RECOVERY_SUPPORT_NUMBER_OF_SESSIONS_RECEIVED':'SvcOtherPeerRecovery','OTHER_PEER_TO_PEER_RECOVERY_SUPPORT_RECEIVED':'SvcOtherPeerRecoverySpec'}, inplace = True)

#Creating variable col4 to replace missing values
col6 = ['SvcInformationReferral','SvcOtherRecoverySupport','SvcOtherPeerRecovery']

#Replacing missing values with 0
df[col6] = df[col6].fillna(0)

In [32]:
#Replacing null values with -9
df['CLIENT_ATTENDED_60%_OR_MORE_OF_PLANNED_SERVICES'] = df['CLIENT_ATTENDED_60%_OR_MORE_OF_PLANNED_SERVICES'].fillna(-9)

#Creating function to create column AttendedMostPlannedServices
def AttendedMost_PlannedServices (x):
    if x == 'Yes':
        return 1
    elif x == 'No':
        return 0
    elif x == -9:
        return -9
    
#Creating column AttendedMostPlannedServices via applying the function AttendedMost_PlannedServices (x)
df['AttendedMostPlannedServices'] = df['CLIENT_ATTENDED_60%_OR_MORE_OF_PLANNED_SERVICES'].apply(lambda x:AttendedMost_PlannedServices (x))

In [33]:
#Replacing null values with -9
df['CLIENT_RECEIVED_SERVICES_VIA_TELEHEALTH_OR_VIRTUAL'] = df['CLIENT_RECEIVED_SERVICES_VIA_TELEHEALTH_OR_VIRTUAL'].fillna(-9)

#Creating function to create column ReceivedTelehealthServices
def Received_Telehealth_Services (x):
    if x == 'Yes':
        return 1
    elif x == 'No':
        return 0
    elif x == -9:
        return -9
    
#Creating column ReceivedTelehealthServices via applying the function Received_Telehealth_Services (x)
df['ReceivedTelehealthServices'] = df['CLIENT_RECEIVED_SERVICES_VIA_TELEHEALTH_OR_VIRTUAL'].apply(lambda x:Received_Telehealth_Services (x))

In [34]:
#Replacing null values with -9
df['CLIENT_PREVIOUSLY_DIAGNOSED_WITH_OPIOID_USE_DISORDER'] = df['CLIENT_PREVIOUSLY_DIAGNOSED_WITH_OPIOID_USE_DISORDER'].fillna(-9)

#Creating a function to create column PreviouslyDiagnosedOUD
def Previously_DiagnosedOUD (row):
    if row['CLIENT_PREVIOUSLY_DIAGNOSED_WITH_OPIOID_USE_DISORDER'] == 'Yes':
        return 1
    elif row['CLIENT_PREVIOUSLY_DIAGNOSED_WITH_OPIOID_USE_DISORDER'] == 'No [SKIP TO QUESTION 5]':
        return 0
    elif row['CLIENT_PREVIOUSLY_DIAGNOSED_WITH_OPIOID_USE_DISORDER'] == -9:
        return -9
    
#Creating column PreviouslyDiagnosedOUD via applying the function Previously_DiagnosedOUD (x)
df['PreviouslyDiagnosedOUD'] = df.apply(Previously_DiagnosedOUD, axis = 1)

In [35]:
#Replacing null values with -9
# df['RECEIVED_MEDICATION_FOR_OPIOID_USE_DISORDER_IN_PAST_30_DAYS'] = df['RECEIVED_MEDICATION_FOR_OPIOID_USE_DISORDER_IN_PAST_30_DAYS'].str.split(',').str[0]
    
#Creating function to create column kOUDMedMethadone
def kOUD_Med_Methadone (row):
    if row ['PreviouslyDiagnosedOUD']== -9 or row ['PreviouslyDiagnosedOUD']== 0 :
        return -1
    elif row['RECEIVED_MEDICATION_FOR_OPIOID_USE_DISORDER_IN_PAST_30_DAYS'] == 'Methadone':
        return 1
    else:
        return 0
    
#Creating column kOUDMedMethadone via applying the function kOUD_Med_Methadone (x)
df['kOUDMedMethadone'] = df.apply(kOUD_Med_Methadone, axis = 1)

In [36]:
# creating column kOUDMedMethadoneDays
df.rename(columns = {'METHADONE_NUMBER_OF_DAYS':'kOUDMedMethadoneDays'}, inplace = True)

def kOUDMedMethadoneDays (row):
    if row['kOUDMedMethadone'] == 1 :
        return row['kOUDMedMethadoneDays']
    else:
        pass
    
df['kOUDMedMethadoneDays'] = df.apply(kOUDMedMethadoneDays, axis = 1)

# data cleaning
# df['kOUDMedMethadoneDays'] = df['kOUDMedMethadoneDays'].apply(lambda x: thirty(x))

In [37]:
#Creating function to create column kOUDMedBuprenorphine
def kOUDMed_Buprenorphine (row):
    if row['PreviouslyDiagnosedOUD'] == -9 or  row ['PreviouslyDiagnosedOUD']== 0 :
        return -1
    elif row['RECEIVED_MEDICATION_FOR_OPIOID_USE_DISORDER_IN_PAST_30_DAYS'] == 'Buprenorphine':
        return 1
    else:
        return 0
    
#Creating column kOUDMedMethadone via applying the function kOUDMed_Buprenorphine (x)
df['kOUDMedBuprenorphine'] = df.apply(kOUDMed_Buprenorphine, axis = 1)

In [38]:
# creating column kOUDMedBuprenorphineDays
df.rename(columns = {'BUPRENORPHINE_NUMBER_OF_DAYS':'kOUDMedBuprenorphineDays'}, inplace = True)

def kOUDMedBuprenorphineDays (row):
    if row['kOUDMedBuprenorphine'] == 1:
        return row['kOUDMedBuprenorphineDays']
    else:
        pass
df['kOUDMedBuprenorphineDays'] = df.apply(kOUDMedBuprenorphineDays, axis = 1)

# data cleaning
# df['kOUDMedBuprenorphineDays'] = df['kOUDMedBuprenorphineDays'].apply(lambda x : thirty(x))

In [39]:
#Creating function to create column kOUDMedNaltrexone
def kOUDMed_Naltrexone (row):
    if row['PreviouslyDiagnosedOUD']== -9 or row ['PreviouslyDiagnosedOUD']== 0 :
        return -1
    elif row['RECEIVED_MEDICATION_FOR_OPIOID_USE_DISORDER_IN_PAST_30_DAYS'] == 'Naltrexone':
        return 1
    else:
        return 0
    
#Creating column kOUDMedNaltrexone via applying the function kOUDMed_Naltrexone (x)
df['kOUDMedNaltrexone'] = df.apply(kOUDMed_Naltrexone, axis = 1)

In [40]:
# creating column kOUDMedNaltrexoneDays
def kOUDMedNaltrexoneDays (row):
    if row['kOUDMedNaltrexone'] == 1:
        return row['NALTREXONE_NUMBER_OF_DAYS']
    else:
        pass

df['kOUDMedNaltrexoneDays'] = df.apply(kOUDMedNaltrexoneDays, axis = 1)

In [41]:
# creating column kOUDMedXRNaltrexone
def kOUDMedXRNaltrexone(row):
    if row['PreviouslyDiagnosedOUD']== -9 or row ['PreviouslyDiagnosedOUD']== 0:
        return -1
    elif row['RECEIVED_MEDICATION_FOR_OPIOID_USE_DISORDER_IN_PAST_30_DAYS'] == 'Extended-release Naltrexone':
            return 1
    else:
            return 0

df['kOUDMedXRNaltrexone'] = df.apply(kOUDMedXRNaltrexone, axis = 1)

In [42]:
# creating column kOUDMedXRNaltrexoneDoses
def kOUDMedXRNaltrexoneDoses(row):
    if row['kOUDMedXRNaltrexone'] == 1:
        return row['NUMBER_OF_DAYS_RECEIVED_EXTENDED_RELEASE_NALTREXONE']
    else:
        pass

df['kOUDMedXRNaltrexoneDoses'] = df.apply(kOUDMedXRNaltrexoneDoses, axis = 1)

# def doses (x):
#     if x > 2:
#         return 2
#     else:
#         pass
# df['kOUDMedXRNaltrexoneDoses'] = df['kOUDMedXRNaltrexoneDoses'].apply(lambda x: doses(x))

In [43]:
# creating the column kOUDMedNotReceived
def kOUDMedNotReceived(row):
    if row['PreviouslyDiagnosedOUD']== -9 or row ['PreviouslyDiagnosedOUD']== 0:
        return -1
    elif row['RECEIVED_MEDICATION_FOR_OPIOID_USE_DISORDER_IN_PAST_30_DAYS'] == 'Client did not receive an FDA-approved medication for a diagnosed opioid use disorder [SKIP TO QUESTION 5]':
        return 1
    else:
        return 0
df['kOUDMedNotReceived'] = df.apply(kOUDMedNotReceived, axis = 1)

In [44]:
# creating column OUDMedTakenAsPrescribed
def OUDMedTakenAsPrescribed (row):
    if row['PreviouslyDiagnosedOUD']== -9 or row ['PreviouslyDiagnosedOUD']== 0:
        return -1
    elif row['kOUDMedMethadone'] == 1:
        return 1
    elif row['kOUDMedBuprenorphine'] == 1:
        return 1
    elif row['kOUDMedNaltrexone'] == 1:
        return 1
    elif row['kOUDMedXRNaltrexone'] == 1:
        return 1
    else:
        return 0
    
df['OUDMedTakenAsPrescribed'] = df.apply(OUDMedTakenAsPrescribed, axis = 1)

In [45]:
df['PREVIOUSLY_DIAGNOSED_WITH_ALCOHOL_USE_DISORDER'] = df['PREVIOUSLY_DIAGNOSED_WITH_ALCOHOL_USE_DISORDER'].fillna(-9)
# creating column PreviouslyDiagnosedAUD
def PreviouslyDiagnosedAUD (row):
    if row['PREVIOUSLY_DIAGNOSED_WITH_ALCOHOL_USE_DISORDER'] == 'Yes':
        return 1
    elif row['PREVIOUSLY_DIAGNOSED_WITH_ALCOHOL_USE_DISORDER'] == 'No':
        return 0
    else:
        return -9
    
df['PreviouslyDiagnosedAUD'] = df.apply(PreviouslyDiagnosedAUD, axis = 1)

In [46]:
# df['RECEIVED_MEDICATION_FOR_ALCOHOL_USE_DISORDER_IN_PAST_30_DAYS'] = df['RECEIVED_MEDICATION_FOR_ALCOHOL_USE_DISORDER_IN_PAST_30_DAYS'].str.split(',').str[0]

def kAUDMedNaltrexone (row):
    if row['PreviouslyDiagnosedAUD'] == -9 or row['PreviouslyDiagnosedAUD'] == 0 :
        return -1
    elif row['RECEIVED_MEDICATION_FOR_ALCOHOL_USE_DISORDER_IN_PAST_30_DAYS'] == 'Naltrexone':
        return 1
    else:
        return 0
    
df['kAUDMedNaltrexone'] = df.apply(kAUDMedNaltrexone, axis = 1)

In [47]:
def kAUDMedNaltrexoneDays (row):
    if row['kAUDMedNaltrexone'] == 1:
        return row['NALTREXONE_NUMBER_OF_DAYS']
    else:
        pass
    
df['kAUDMedNaltrexoneDays'] = df.apply(kAUDMedNaltrexoneDays, axis = 1)

In [48]:
def kAUDMedXRNaltrexone (row):
    if row['PreviouslyDiagnosedAUD'] == -9 or row['PreviouslyDiagnosedAUD'] == 0:
        return -1
    elif row['RECEIVED_MEDICATION_FOR_ALCOHOL_USE_DISORDER_IN_PAST_30_DAYS'] == 'Extended-release naltrexone':
        return 1
    else:
        return 0
    
df['kAUDMedXRNaltrexone'] = df.apply(kAUDMedXRNaltrexone, axis = 1)

In [49]:
def kAUDMedXRNaltrexoneDoses (row):
    if row['kAUDMedXRNaltrexone'] == 1:
        return row['EXTENDED_RELEASE_NALTREXONE_NUMBER_OF_DAYS']
    else:
        pass

df['kAUDMedXRNaltrexoneDoses'] = df.apply(kAUDMedXRNaltrexone, axis = 1)

In [50]:
def kAUDMedDisulfiram (row):
    if row['PreviouslyDiagnosedAUD'] == -9 or row['PreviouslyDiagnosedAUD'] == 0:
        return -1
    elif row['RECEIVED_MEDICATION_FOR_ALCOHOL_USE_DISORDER_IN_PAST_30_DAYS'] == 'Disulfiram':
        return 1
    else:
        return 0
    
df['kAUDMedDisulfiram'] = df.apply(kAUDMedDisulfiram , axis = 1)

In [51]:
def kAUDMedDisulfiramDays (row):
    if row['kAUDMedDisulfiram'] == 1:
        return row['NUMBER_OF_DAYS_RECEIVED_DISULFIRAM']
    else:
        pass
df['kAUDMedDisulfiramDays'] = df.apply(kAUDMedDisulfiramDays, axis = 1)

In [52]:
def kAUDMedAcamprosate (row):
    if row['PreviouslyDiagnosedAUD'] == -9 or row['PreviouslyDiagnosedAUD'] == 0:
        return -1
    elif row['RECEIVED_MEDICATION_FOR_ALCOHOL_USE_DISORDER_IN_PAST_30_DAYS'] == 'Acamprosate':
        return 1
    else:
        return 0
df['kAUDMedAcamprosate'] = df.apply(kAUDMedAcamprosate, axis = 1)

In [53]:
def kAUDMedAcamprosateDays (row):
    if row['kAUDMedAcamprosate'] == 1:
        return row['NUMBER_OF_DAYS_RECEIVED_ACAMPROSATE']
    else:
        pass
df['kAUDMedAcamprosateDays'] = df.apply(kAUDMedAcamprosateDays, axis = 1)

In [54]:
#Creating function to create column kAUDMedNotReceived
def  kAUDMed_NotReceived (row):
    if row['PreviouslyDiagnosedAUD'] == -9 or row['PreviouslyDiagnosedAUD'] == 0:
        return -1
    elif row['RECEIVED_MEDICATION_FOR_ALCOHOL_USE_DISORDER_IN_PAST_30_DAYS'] == 'Client did not receive an FDA-approved medication for an alcohol use disorder [SKIP TO QUESTION 6]':
        return 1
    else:
        return 0
    
#Creating column kAUDMedNotReceived via applying the function kAUDMed_NotReceived (x)
df['kAUDMedNotReceived'] = df.apply(kAUDMed_NotReceived, axis = 1)

In [55]:
#Creating column with missing values
def AUDMedTakenAsPrescribed (row):
    if row['PreviouslyDiagnosedAUD'] == -9 or row['PreviouslyDiagnosedAUD'] == 0:
        return -1
    elif row['kAUDMedNaltrexone'] == 1:
        return 1
    elif row['kAUDMedXRNaltrexone'] == 1:
        return 1
    elif row['kAUDMedDisulfiram'] == 1:
        return 1
    elif row['kAUDMedAcamprosate'] == 1:
        return 1
    else:
        return 0
    
df['AUDMedTakenAsPrescribed'] = df.apply(AUDMedTakenAsPrescribed, axis = 1)

In [56]:
#Replacing null values with -9
df['PREVIOUSLY_DIAGNOSED_WITH_STIMULANT_USE_DISORDER'] = df['PREVIOUSLY_DIAGNOSED_WITH_STIMULANT_USE_DISORDER'].fillna(-9)

#Creating function to create column PreviouslyDiagnosedStUD
def Previously_DiagnosedStUD (row):
    if row['PREVIOUSLY_DIAGNOSED_WITH_STIMULANT_USE_DISORDER'] == 'Yes':
        return 1
    elif row['PREVIOUSLY_DIAGNOSED_WITH_STIMULANT_USE_DISORDER'] == 'No [SKIP TO QUESTION 7]':
        return 0
    elif row['PREVIOUSLY_DIAGNOSED_WITH_STIMULANT_USE_DISORDER'] == -9:
        return -9
    
#Creating column PreviouslyDiagnosedStUD via applying the function Previously_DiagnosedStUD (x)
df['PreviouslyDiagnosedStUD'] = df.apply(Previously_DiagnosedStUD, axis = 1)

In [57]:
#Replacing null valules with -9
# df['INTERVENTIONS_RECEIVED_FOR_STIMULANT_USE_DISORDER_IN_PAST_30_DAYS'] = df['INTERVENTIONS_RECEIVED_FOR_STIMULANT_USE_DISORDER_IN_PAST_30_DAYS'].str.split(',').str[0]

#Creating function to create column kStUDIntContMgmt
def kStUDIntContMgmt (row):
    if row['PreviouslyDiagnosedStUD'] == -9 or row['PreviouslyDiagnosedStUD'] == 0 :
        return -1
    elif row['INTERVENTIONS_RECEIVED_FOR_STIMULANT_USE_DISORDER_IN_PAST_30_DAYS'] == 'Contingency Management':
        return 1
    else:
        return 0

#Creating column kStUDIntCommReinf via applying the function kStUDInt_CommReinf (x)
df['kStUDIntContMgmt'] = df.apply(kStUDIntContMgmt, axis = 1)

In [58]:
def kStUDIntContMgmtDays (row):
    if row['kStUDIntContMgmt'] == 1:
        return row['CONTINGENCY_MANAGEMENT_NUMBER_OF_DAYS']
    else:
        pass
df['kStUDIntContMgmtDays'] = df.apply(kStUDIntContMgmtDays, axis = 1)

In [59]:
#Creating function to create column kStUDIntCommReinf
def kStUDIntCommReinf (row):
    if row['PreviouslyDiagnosedStUD'] == -9 or row['PreviouslyDiagnosedStUD'] == 0:
        return -1
    elif row['INTERVENTIONS_RECEIVED_FOR_STIMULANT_USE_DISORDER_IN_PAST_30_DAYS'] == 'Community Reinforcement':
        return 1
    else:
        return 0

#Creating column kStUDIntCommReinf via applying the function kkStUDIntCommReinf (x)
df['kStUDIntCommReinf'] = df.apply(kStUDIntCommReinf, axis = 1)

In [60]:
def kStUDIntCommReinfDays (row):
    if row['kStUDIntCommReinf'] == 1:
        return row['COMMUNITY_REINFORCEMENT_NUMBER_OF_DAYS']
    else:
        pass
df['kStUDIntCommReinfDays'] = df.apply(kStUDIntCommReinfDays, axis = 1)

In [61]:
def kStUDIntCBT (row):
    if row['PreviouslyDiagnosedStUD'] == -9 or row['PreviouslyDiagnosedStUD'] == 0:
        return -1
    elif row['INTERVENTIONS_RECEIVED_FOR_STIMULANT_USE_DISORDER_IN_PAST_30_DAYS'] == 'Cognitive Behavioral Therapy':
        return 1 
    else:
        return 0
df['kStUDIntCBT'] = df.apply(kStUDIntCBT, axis = 1)

In [62]:
def kStUDIntCBTDays (row):
    if row['kStUDIntCBT'] == 1 :
        return row['COGNITIVE_BEHAVIORAL_THERAPY_NUMBER_OF_DAYS'] 
    else:
        pass
df['kStUDIntCBTDays'] = df.apply(kStUDIntCBTDays, axis = 1)

In [63]:
#Creating function to create column kStUDIntOther
def kStUDIntOther (row):
    if row['PreviouslyDiagnosedStUD'] == -9 or row['PreviouslyDiagnosedStUD'] == 0:
        return -1
    elif row['INTERVENTIONS_RECEIVED_FOR_STIMULANT_USE_DISORDER_IN_PAST_30_DAYS'] == 'Other treatment approach':
        return 1
    else:
        return 0

#Creating column kStUDIntOther via applying the functionkStUD_IntOther (x)
df['kStUDIntOther'] = df.apply(kStUDIntOther, axis = 1)

In [64]:
def kStUDIntOtherDays (row):
    if row['kStUDIntOther'] == 1:
        return row['OTHER_TREATMENT_NUMBER_OF_DAYS']
    else:
        pass
df['kStUDIntOtherDays'] = df.apply(kStUDIntOtherDays, axis = 1)

In [65]:
#Creating function to create column kStUDIntNotReceived
def kStUDIntNotReceived (row):
    if row['PreviouslyDiagnosedStUD'] == -9 or row['PreviouslyDiagnosedStUD'] == 0:
        return -1 
    elif row['INTERVENTIONS_RECEIVED_FOR_STIMULANT_USE_DISORDER_IN_PAST_30_DAYS'] == 'Client did not receive any intervention for a stimulant use disorder [SKIP TO QUESTION 7]':
        return 1
    else:
        return 0

#Creating column kStUDIntNotReceived via applying the function kStUD_IntNotReceived (x)
df['kStUDIntNotReceived'] = df.apply(kStUDIntNotReceived, axis = 1)

In [66]:
def StUDIntAttended (row):
    if row['PreviouslyDiagnosedStUD'] == -9 or row['PreviouslyDiagnosedStUD'] == 0:
        return -1
    elif row['kStUDIntContMgmt'] == 1:
        return 1
    elif row['kStUDIntCommReinf'] == 1:
        return 1
    elif row['kStUDIntCBT'] == 1:
        return 1
    elif row['kStUDIntOther'] == 1:
        return 1
    else:
        return 0
    
df['StUDIntAttended'] = df.apply( StUDIntAttended, axis = 1)

In [67]:
#Replacing null values with -9
df['PREVIOUSLY_DIAGNOSED_WITH_TOBACCO_USE_DISORDER'] = df['PREVIOUSLY_DIAGNOSED_WITH_TOBACCO_USE_DISORDER'].fillna(-9)

#Creating function to create column PreviouslyDiagnosedTUD
def PreviouslyDiagnosedTUD (row):
    if row['PREVIOUSLY_DIAGNOSED_WITH_TOBACCO_USE_DISORDER'] == 'Yes':
        return 1
    elif row['PREVIOUSLY_DIAGNOSED_WITH_TOBACCO_USE_DISORDER'] == 'No [THE DISCHARGE INTERVIEW IS COMPLETE.]':
        return 0
    elif row['PREVIOUSLY_DIAGNOSED_WITH_TOBACCO_USE_DISORDER'] == -9:
        return -9
    
#Creating column PreviouslyDiagnosedTUD via applying the function 
df['PreviouslyDiagnosedTUD'] = df.apply(PreviouslyDiagnosedTUD, axis = 1)

In [68]:
# df['MEDICATIONS_RECEIVED_FOR_TOBACCO_USE_DISORDER_IN_PAST_30_DAYS'] = df['MEDICATIONS_RECEIVED_FOR_TOBACCO_USE_DISORDER_IN_PAST_30_DAYS'].str.split(',').str[0]

def kTUDMedNicotineRepl (row):
    if row['PreviouslyDiagnosedTUD'] == -9 or row['PreviouslyDiagnosedTUD'] == 0:
        return -1
    elif row['MEDICATIONS_RECEIVED_FOR_TOBACCO_USE_DISORDER_IN_PAST_30_DAYS'] == 'Nicotine Replacement':
        return 1
    else:
        return 0  
df['kTUDMedNicotineRepl'] = df.apply(kTUDMedNicotineRepl, axis =1 )

In [69]:
def kTUDMedNicotineReplDays (row):
    if row['kTUDMedNicotineRepl'] == 1:
        return row['NICOTINE_REPLACEMENT_NUMBER_OF_DAYS']
    else:
        pass
df['kTUDMedNicotineReplDays'] = df.apply(kTUDMedNicotineReplDays, axis = 1)

In [70]:
def kTUDMedBupropion (row):
    if row['PreviouslyDiagnosedTUD'] == -9 or row['PreviouslyDiagnosedTUD'] == 0:
        return -1
    elif row['MEDICATIONS_RECEIVED_FOR_TOBACCO_USE_DISORDER_IN_PAST_30_DAYS'] == 'Bupropion':
        return 1
    else:
        return 0
df['kTUDMedBupropion'] = df.apply(kTUDMedBupropion, axis = 1)

In [71]:
def kTUDMedBupropionDays (row):
    if row['kTUDMedBupropion'] == 1:
        return row['BUPROPION_NUMBER_OF_DAYS']
    else:
        pass
df['kTUDMedBupropionDays'] = df.apply(kTUDMedBupropionDays, axis = 1)

In [72]:
def kTUDMedVarenicline (row):
    if row['PreviouslyDiagnosedTUD'] == -9 or row['PreviouslyDiagnosedTUD'] == 0:
        return -1
    elif row ['MEDICATIONS_RECEIVED_FOR_TOBACCO_USE_DISORDER_IN_PAST_30_DAYS'] == 'Varenicline':
        return 1
    else:
        return 0
    
df['kTUDMedVarenicline'] = df.apply(kTUDMedVarenicline, axis = 1)

In [73]:
def kTUDMedVareniclineDays (row):
    if row['kTUDMedVarenicline'] == 1:
        return row['VARENICLINE_NUMBER_OF_DAYS']
    else:
        pass
df['kTUDMedVareniclineDays'] = df.apply(kTUDMedVareniclineDays, axis = 1)

In [74]:
def kTUDMedNotReceived (row):
    if row['PreviouslyDiagnosedTUD'] == -9 or row['PreviouslyDiagnosedTUD'] == 0:
        return -1
    elif row['MEDICATIONS_RECEIVED_FOR_TOBACCO_USE_DISORDER_IN_PAST_30_DAYS'] == 'Client did not receive an FDA-approved medication for a tobacco use disorder [THE DISCHARGE INTERVIEW IS COMPLETE.]':
        return 1
    else:
        return 0
df['kTUDMedNotReceived'] = df.apply(kTUDMedNotReceived, axis = 1)

In [75]:
def TUDMedTakenAsPrescribed (row):
    if row['PreviouslyDiagnosedTUD'] == -9 or row['PreviouslyDiagnosedTUD'] == 0:
        return -1
    elif row['kTUDMedNicotineRepl'] == 1:
        return 1
    elif row['kTUDMedBupropion'] == 1:
        return 1
    elif row['kTUDMedVarenicline'] == 1:
        return 1
    else:
        return 0
df['TUDMedTakenAsPrescribed'] = df.apply(TUDMedTakenAsPrescribed, axis = 1)

In [76]:
#Creating the IntakeDate and InterviewCreateDate columns 
now = dt.now().strftime('%m/%d/%Y')

df['InterviewCreateDate'] = now

In [77]:
#Renaming column to create column SvcRecoveryPlanning
df.rename(columns = {'RECOVERY_PLANNING_NUMBER_OF_SESSIONS_RECEIVED_1':'SvcRecoveryPlanning'},inplace = True)

#Replacing null values with 0
df['SvcRecoveryPlanning'] = df['SvcRecoveryPlanning'].fillna(0)

In [78]:
#Inserting blank columns from section H

section_h_blank_col = [
'ReunitedChild',
'ReunitedChildWithAgcySupe',
'ReunitedChildWithoutAgcySupe',
'AvoidedOutOfHomePlacement',
'H1NoneOfAbove',
'HelpObPrivateHealthInsurance',
'HelpObMedicaid',
'HelpObMedicare',
'HelpObSSIOrSSDI',
'HelpObTANF',
'HelpObSNAP',
'HelpObOtherBenefit',
'HelpObOtherBenefitSpec',
'HelpObNoneOfTheAbove',
'EnrolledInSchool',
'EnrolledInSchoolHelp',
'EnrolledInVocTraining',
'EnrolledInVocTrainingHelp',
'CurrentlyEmployed',
'CurrentlyEmployedHelp',
'LivingInStableHousing',
'LivingInStableHousingHelp',
'MaintainFamilyResponsibilities',
'SupportToParentingAndRecovery',
'EnableFocusOnTreatment',
'SupportToParentingAndRecoveryH5',
'SexActivity',
'SexContacts',
'SexUnprot',
'SexUnprotectedHIV',
'SexUnprotectedInjDrugUser',
'SexUnprotectedHigh',
'PrEPOrTreat',
'HIVTestH',
'HIVTestFirstTime',
'HIVTestLocation',
'HIVTestReferralLocation',
'HIVTestReferralLocationSpec',
'HIVTestResultH',
'HIVTestConfirmatory',
'HIVTestConfirmatoryRes',
'TreatmentForHIVH',
'TreatmentForHIVLocation',
'TreatmentForHIVLocationSpec',
'HIVTestingPartners',
'HIVTestingPartnersNumber',
'HIVReferredPreEP',
'HIVReferredPEP',
'HIVReferredCounseling',
'HIVReferredNoMedications',
'HIVReferredNoCounseling',
'HCVTest',
'HCVTestConfirmatory',
'HCVTestResult',
'TreatmentForHCV',
'HBVTest',
'HBVTestResult',
'TreatmentForHBV',
'HepABVacc',
'HepABVaccReferral',
'PeerSupport',
'EnrolledInSchoolH8',
'EnrolledInSchoolHelpH8',
'EnrolledInVocTrainingH8',
'EnrolledInVocTrainingHelpH8',
'CurrentlyEmployedH8',
'CurrentlyEmployedHelpH8',
'LivingInStableHousingH8',
'LivingInStableHousingHelpH8',
'QualityOfLife',
'CommunicateWithProvider',
'ReduceSubstanceUse',
'ManageMentalHealthSymptoms',
'SupportRecovery',
'MentalHealthDisorderScreen',
'MentalHealthServiceReferred',
'MentalHealthServiceReceived',
'CoOccurringDisorderScreen',
'CoOccurringServiceReferred',
'CoOccurringServiceReceived',
'HelpedAvoidJusticeSystem'

]

df[section_h_blank_col] = df[section_h_blank_col] = ''

In [79]:
# # creating list object for columns that will need to be empty for the file submission
# filter_columns = ['jHIVTest',	'jHIVTestResult',	'ODPreventionToolsProvided',
#                   'VaccinatedStatusCOVID19',	'SvcCaseManagement',	'SvcIntensiveOutpatient',
#                   'SvcInpatient',	'SvcOutpatient',	'SvcOutreach',	'SvcMethadone',	'SvcBuprenorphine',
#                   'SvcNaltrexoneShortActing',	'SvcNaltrexoneLongActing',	'SvcDisulfiram',
#                   'SvcAcamprosate',	'SvcNicotineReplacement',	'SvcBupropion',	'SvcVarenicline',
#                   'SvcResidentialRehab',	'SvcHospitalInpatient',	'SvcFreeStandingRes',	'SvcAmbulatoryDetox',
#                   'SvcAfterCare',	'SvcRecoverySupport',	'SvcOtherModalities',	'SvcOtherModalitesSpec',
#                   'SvcScreening',	'SvcBriefIntervention',	'SvcBriefTreatment',	'SvcReferralTreatment',
#                   'SvcAssessment',	'SvcTreatmentPlanning',	'SvcRecoveryPlanning',	'SvcIndividualCouns',
#                   'SvcGroupCouns',	'SvcContingencyManagement',	'SvcCommunityReinforcement',	'SvcCBT',
#                   'SvcFamilyMarriageCouns',	'SvcCoOccurring',	'SvcPharmacological',	'SvcHIVAIDSCouns',
#                   'SvcCulturalInterventions',	'SvcOtherClinicalCouns',	'SvcOtherClinicalCounsSpec',
#                   'SvcFamilyServices',	'SvcChildCare',	'SvcPreEmployment',	'SvcEmploymentCoaching',
#                   'SvcIndividualCoord',	'SvcTransportation',	'SvcHIVAIDSServices',	'SvcDrugFreeHousing',
#                   'SvcHousingSupport',	'SvcHealthInsuranceEnrollment',	'SvcOtherCaseMgmt',	'SvcOtherCaseMgmtSpec',
#                   'SvcMedicalCare',	'SvcAlcoholDrugTesting',	'SvcOBGYN',	'SvcHIVAIDSMedical',
#                   'SvcHepatitisSupport',	'SvcOtherSTISupport',	'SvcDental',	'SvcOtherMedical',
#                   'SvcOtherMedicalSpec',	'SvcContinuingCare',	'SvcRelapsePrevention',	'SvcRecoveryCoaching',
#                   'SvcSelfHelpSupport',	'SvcSpiritualSupport',	'SvcOtherAfterCare',	'SvcOtherAfterCareSpec',
#                   'SvcSubstanceAbuseEdu',	'SvcHIVAIDSEdu',	'SvcHepatitisEdu',	'SvcOtherSTIEdu',
#                   'SvcNaloxoneTraining',	'SvcFentanylTestStripTraining',	'SvcOtherEdu',	'SvcOtherEduSpec',
#                   'SvcPeerCoaching',	'SvcVocational',	'SvcRecoveryHousing',	'SvcRecoveryPlanningRSS',
#                   'SvcRecoveryCaseManagement',	'SvcDrugFreeSocial',	'SvcInformationReferral',
#                   'SvcOtherRecoverySupport',	'SvcOtherRecoverySupportSpec',	'SvcOtherPeerRecovery',
#                   'SvcOtherPeerRecoverySpec',	'AttendedMostPlannedServices',	'ReceivedTelehealthServices',
#                   'PreviouslyDiagnosedOUD',	'kOUDMedMethadone',	'kOUDMedMethadoneDays',	'kOUDMedBuprenorphine',
#                   'kOUDMedBuprenorphineDays',	'kOUDMedNaltrexone',	'kOUDMedNaltrexoneDays',
#                   'kOUDMedXRNaltrexone',	'kOUDMedXRNaltrexoneDoses',	'kOUDMedNotReceived',
#                   'OUDMedTakenAsPrescribed',	'PreviouslyDiagnosedAUD',	'kAUDMedNaltrexone',
#                   'kAUDMedNaltrexoneDays',	'kAUDMedXRNaltrexone',	'kAUDMedXRNaltrexoneDoses',
#                   'kAUDMedDisulfiram',	'kAUDMedDisulfiramDays',	'kAUDMedAcamprosate',
#                   'kAUDMedAcamprosateDays',	'kAUDMedNotReceived',	'AUDMedTakenAsPrescribed',
#                   'PreviouslyDiagnosedStUD',	'kStUDIntContMgmt',	'kStUDIntContMgmtDays',
#                   'kStUDIntCommReinf',	'kStUDIntCommReinfDays',	'kStUDIntCBT',
#                   'kStUDIntCBTDays',	'kStUDIntOther',	'kStUDIntOtherDays',
#                   'kStUDIntNotReceived',	'StUDIntAttended',	'PreviouslyDiagnosedTUD',
#                   'kTUDMedNicotineRepl',	'kTUDMedNicotineReplDays',	'kTUDMedBupropion',
#                   'kTUDMedBupropionDays',	'kTUDMedVarenicline',	'kTUDMedVareniclineDays',
#                   'kTUDMedNotReceived',	'TUDMedTakenAsPrescribed'
#                  ]

# # function to return values based on interview type --> excluding responses for Intakes and Follow-Ups
# df[filter_columns] = ''

In [80]:
#Creating QAOnly column
df['QAOnly'] = ''

## Exporting GPRA 6

In [82]:
#Creating variable to export GPRA 6
GPRA6_SBIRT = df[['ClientID','FLWPClientContacted',	'FLWPStatus',	'FLWPStatusSpec',	'ReceivingServices',
                  'DischargeDate',	'DischargeStatusCompl',	'DischargeStatusTermReason',	'OtherDischargeStatTermRsnSpec',
                  'jHIVTest',	'jHIVTestResult',	'ODPreventionToolsProvided',	'VaccinatedStatusCOVID19',
                  'SvcCaseManagement',	'SvcIntensiveOutpatient',	'SvcInpatient',	'SvcOutpatient',	'SvcOutreach',
                  'SvcMethadone',	'SvcBuprenorphine',	'SvcNaltrexoneShortActing',	'SvcNaltrexoneLongActing',
                  'SvcDisulfiram',	'SvcAcamprosate',	'SvcNicotineReplacement',	'SvcBupropion',	'SvcVarenicline',
                  'SvcResidentialRehab',	'SvcHospitalInpatient',	'SvcFreeStandingRes',	'SvcAmbulatoryDetox',
                  'SvcAfterCare',	'SvcRecoverySupport',	'SvcOtherModalities',	'SvcOtherModalitesSpec',
                  'SvcScreening',	'SvcBriefIntervention',	'SvcBriefTreatment',	'SvcReferralTreatment',	
                  'SvcAssessment',	'SvcTreatmentPlanning',	'SvcRecoveryPlanning',	'SvcIndividualCouns',
                  'SvcGroupCouns',	'SvcContingencyManagement',	'SvcCommunityReinforcement',	'SvcCBT',
                  'SvcFamilyMarriageCouns',	'SvcCoOccurring',	'SvcPharmacological',	'SvcHIVAIDSCouns',
                  'SvcCulturalInterventions',	'SvcOtherClinicalCouns',	'SvcOtherClinicalCounsSpec',
                  'SvcFamilyServices',	'SvcChildCare',	'SvcPreEmployment',	'SvcEmploymentCoaching',	'SvcIndividualCoord',
                  'SvcTransportation',	'SvcHIVAIDSServices',	'SvcDrugFreeHousing',	'SvcHousingSupport',
                  'SvcHealthInsuranceEnrollment',	'SvcOtherCaseMgmt',	'SvcOtherCaseMgmtSpec',	'SvcMedicalCare',
                  'SvcAlcoholDrugTesting',	'SvcOBGYN',	'SvcHIVAIDSMedical',	'SvcHepatitisSupport',
                  'SvcOtherSTISupport',	'SvcDental','SvcOtherMedical','SvcOtherMedicalSpec',	'SvcContinuingCare',
                  'SvcRelapsePrevention',	'SvcRecoveryCoaching',	'SvcSelfHelpSupport',	'SvcSpiritualSupport',
                  'SvcOtherAfterCare',	'SvcOtherAfterCareSpec',	'SvcSubstanceAbuseEdu',	'SvcHIVAIDSEdu',
                  'SvcHepatitisEdu',	'SvcOtherSTIEdu',	'SvcNaloxoneTraining',	'SvcFentanylTestStripTraining',
                  'SvcOtherEdu',	'SvcOtherEduSpec',	'SvcPeerCoaching',	'SvcVocational',	'SvcRecoveryHousing',
                  'SvcRecoveryPlanningRSS',	'SvcRecoveryCaseManagement',	'SvcDrugFreeSocial',
                  'SvcInformationReferral',	'SvcOtherRecoverySupport',	'SvcOtherRecoverySupportSpec',
                  'SvcOtherPeerRecovery',	'SvcOtherPeerRecoverySpec',	'AttendedMostPlannedServices',
                  'ReceivedTelehealthServices',	'PreviouslyDiagnosedOUD',	'kOUDMedMethadone',	'kOUDMedMethadoneDays',	
                  'kOUDMedBuprenorphine',	'kOUDMedBuprenorphineDays',	'kOUDMedNaltrexone',	'kOUDMedNaltrexoneDays',
                  'kOUDMedXRNaltrexone','kOUDMedXRNaltrexoneDoses',	'kOUDMedNotReceived',	'OUDMedTakenAsPrescribed',
                  'PreviouslyDiagnosedAUD',	'kAUDMedNaltrexone',	'kAUDMedNaltrexoneDays',	'kAUDMedXRNaltrexone',
                  'kAUDMedXRNaltrexoneDoses',	'kAUDMedDisulfiram',	'kAUDMedDisulfiramDays',	'kAUDMedAcamprosate',
                  'kAUDMedAcamprosateDays',	'kAUDMedNotReceived',	'AUDMedTakenAsPrescribed',	'PreviouslyDiagnosedStUD',
                  'kStUDIntContMgmt',	'kStUDIntContMgmtDays',	'kStUDIntCommReinf',	'kStUDIntCommReinfDays',
                  'kStUDIntCBT',	'kStUDIntCBTDays',	'kStUDIntOther',	'kStUDIntOtherDays',	'kStUDIntNotReceived',
                  'StUDIntAttended',	'PreviouslyDiagnosedTUD',	'kTUDMedNicotineRepl',	'kTUDMedNicotineReplDays',
                  'kTUDMedBupropion',	'kTUDMedBupropionDays',	'kTUDMedVarenicline',	'kTUDMedVareniclineDays',
                  'kTUDMedNotReceived',	'TUDMedTakenAsPrescribed','ReunitedChild',	'ReunitedChildWithAgcySupe',
                  'ReunitedChildWithoutAgcySupe',	'AvoidedOutOfHomePlacement',	'H1NoneOfAbove',
                  'HelpObPrivateHealthInsurance',	'HelpObMedicaid',	'HelpObMedicare',	'HelpObSSIOrSSDI',
                  'HelpObTANF',	'HelpObSNAP',	'HelpObOtherBenefit',	'HelpObOtherBenefitSpec',	'HelpObNoneOfTheAbove',
                  'EnrolledInSchool',	'EnrolledInSchoolHelp',	'EnrolledInVocTraining',	'EnrolledInVocTrainingHelp',
                  'CurrentlyEmployed',	'CurrentlyEmployedHelp',	'LivingInStableHousing',	'LivingInStableHousingHelp',
                  'MaintainFamilyResponsibilities',	'SupportToParentingAndRecovery',	'EnableFocusOnTreatment',
                  'SupportToParentingAndRecoveryH5',	'SexActivity',	'SexContacts',	'SexUnprot',	
                  'SexUnprotectedHIV',	'SexUnprotectedInjDrugUser',	'SexUnprotectedHigh',	'PrEPOrTreat',
                  'HIVTestH',	'HIVTestFirstTime',	'HIVTestLocation',	'HIVTestReferralLocation',
                  'HIVTestReferralLocationSpec',	'HIVTestResultH',	'HIVTestConfirmatory',	
                  'HIVTestConfirmatoryRes',	'TreatmentForHIVH',	'TreatmentForHIVLocation',	'TreatmentForHIVLocationSpec',
                  'HIVTestingPartners',	'HIVTestingPartnersNumber',	'HIVReferredPreEP',	'HIVReferredPEP',	
                  'HIVReferredCounseling',	'HIVReferredNoMedications',	'HIVReferredNoCounseling',	'HCVTest',
                  'HCVTestConfirmatory',	'HCVTestResult',	'TreatmentForHCV',	'HBVTest',	'HBVTestResult',
                  'TreatmentForHBV',	'HepABVacc',	'HepABVaccReferral',	'PeerSupport',	'EnrolledInSchoolH8',
                  'EnrolledInSchoolHelpH8',	'EnrolledInVocTrainingH8',	'EnrolledInVocTrainingHelpH8',	'CurrentlyEmployedH8',
                  'CurrentlyEmployedHelpH8',	'LivingInStableHousingH8',	'LivingInStableHousingHelpH8',	'QualityOfLife',
                  'CommunicateWithProvider',	'ReduceSubstanceUse',	'ManageMentalHealthSymptoms',	'SupportRecovery',
                  'MentalHealthDisorderScreen',	'MentalHealthServiceReferred',	'MentalHealthServiceReceived',	'CoOccurringDisorderScreen',
                  'CoOccurringServiceReferred',	'CoOccurringServiceReceived',	'HelpedAvoidJusticeSystem','InterviewCreateDate','QAOnly']]


#Exporting GPRA 6
GPRA6_SBIRT.to_csv('GPRA6_SBIRT.csv', index = False, header = True, encoding = 'utf-8')

